# Multi-Step Tool Use - Calender Assistant

Multi-step tool use happens when more than one tool is required, and the output of one tool is needed as the input to another tool.

In other words, tool-calling needs to happen in a sequence.

This is especially true in complex workflows where tasks have to be broken down into a sequence of steps.

In this notebook, we’ll build a calendar assistant that can check a user’s calendar and create new appointments based on available times. This use case requires tool calls to happen in sequence, i.e., multi-step.

It is also an example of using tools for performing not just a read operation (retrieving information) but also a write operation (taking actions).


First, let’s install the AWS Cohere Python SDK and set up the Bedrock client.

In [ ]:
!pip install --upgrade setuptools==69.5.1 wheel --quiet
!pip install cohere -q

In [ ]:
import json
import cohere_aws

In [ ]:
co= cohere_aws.Client(mode=cohere_aws.Mode.BEDROCK)
model = "cohere.command-r-plus-v1:0"

# Create Tools

Let’s now create two tools:
- A function called `list_calendar_events` to list the existing calendar events based on a given date. For simplicity, we are not going to connect to an actual calendar. So we’ll include a mock events record, which is a simple list of events in a day.
- A function called `create_calendar_event` to create a new calendar event based on the provided date, time, and duration. Again, to keep things simple, we are not going to make actual changes to a database, but instead will just return a simple success message for illustration purposes.

In [ ]:
# Define the tools

def list_calendar_events(date: str):
  events = [{"start": "8:00", "end": "8:59"}, {"start": "9:00", "end": "9:59"}, {"start": "11:00", "end": "11:59"},{"start": "12:00", "end": "12:59"}]

  return {
        "existing_events": events
    }

def create_calendar_event(date: str, time: str, duration: int):
  
  return {
        "is_success": True,
        "message": f"Created a {duration} hour long event at {time} on {date}"
    }

functions_map = {
    "list_calendar_events": list_calendar_events,
    "create_calendar_event": create_calendar_event
}


Next, we define the tool schema for the two tools.

In [ ]:
tools = [
    {
      "name": "list_calendar_events",
      "description": "Returns a list of existing events for the specified date, including the start time and end time for each event.",
      "parameter_definitions": {
        "date": {
          "description": "the date to list events for, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        }
      }
    }, 
    {
      "name": "create_calendar_event",
      "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
      "parameter_definitions": {
        "date": {
          "description": "the date on which the event starts, formatted as mm/dd/yy",
          "type": "str",
          "required": True
        },
        "time": {
          "description": "the time of the event, formatted using 24h military time formatting",
          "type": "str",
          "required": True
        },
        "duration": {
          "description": "the number of hours the event lasts for",
          "type": "float",
          "required": True
        }
      }
    }
]

Let’s also create a custom preamble. The important part here is mentioning the date since the `list_calendar_events` tool is a simple enough version that it doesn’t do any date-handling. What we’re doing is nudging the model to use this date for the create_calendar_event tool call, which requires a date field.

In [ ]:
preamble="""## Task & Context
You are a calendar assistant who helps people schedule events on their calendar. You must make sure that a new event does not overlap with any existing event.
Today is Thursday, May 23, 2024
"""

Let's create a `run_assistant` function that does the following:
- Get the user message (Step 1)
- Call the Chat endpoint for tool call generation (Step 2)
- If the response contains at least one tool call, execute the tool call(s) and get the tool results (Step 3)
- Repeat Steps 2 and 3 until there are no more tool calls
- Generate the final response with citations (Step 4)

Here, we remove the `force_single_step` argument as it is `False` by default. This indicates to the Cohere API to enable a multi-step workflow and use the right ReAct-style prompt for running the workflow.

In [ ]:
def run_assistant(message, chat_history=None):
    if chat_history is None:
        chat_history = []
        
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)          
    
    response = co.chat(
        message=message,
        model_id=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model_id=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

## Question that doesn't require a tool

In [ ]:
chat_history = run_assistant("Give me 3 concise tips on how to build a great company")

## Question that selects a tool with incomplete data to answer question


In [ ]:
chat_history = run_assistant("Do I have any important appointments lined up in September")

## Simple question that doesn't require multi-step (single step)

Let’s now ask the assistant the first question, starting with a simple one about the number of meetings for the day. This requires just a single step of tool calling to check existing calendar events.

That’s exactly what the assistant does, and it gives the correct answer.

In [ ]:
chat_history = run_assistant("How many meetings do I have today")

In [ ]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

## Question that require's multi-step


Let’s now try to ask a question that requires multi-step tool calling — this one asking the assistant to book an appointment. To complete this task, the assistant will have to first query the existing events and then use the information to create a new event that wouldn’t cause a conflict.

In the first step, the assistant calls the `list_calendar_events` tool to get a list of existing events.

In the second step, it calls the `create_calendar_event` tool to create a new event. It creates a new event at 10 a.m., which is indeed the first available slot after 9 a.m.

In [ ]:
chat_history = run_assistant("Create an hour-long appointment for the first available free slot after 9am")

And here’s a look at the chat history. Compared to the single-step scenario, it contains multiple pairs of CHATBOT - TOOL messages, with each pair being one step in the sequence of tool calls. In this particular case, we have two of them.

In [ ]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

The model has “intermediate response” to differentiate it from the model’s final response, which is the response that the user sees. These intermediate responses are the model’s internal reasoning logic, which guides its next course of action.

## Multi-step parallel

Let’s now look at an example of multi-step, parallel tool use. Let’s ask the assistant to create two separate events on the same day.

And here’s the assistant’s response. It follows the same two steps as in the earlier example: first, checking the existing events, and second, creating the new events.

The difference this time is that since it needs to create two events, it calls `create_calendar_event` twice within the same step. Here the assistant makes the correct judgment and tool calls are independent of each other, so they can be done in parallel.

In [ ]:
chat_history = run_assistant("Create two hour-long appointments for any available time between 8am to 6pm")

## State management (memory)

The chat history for each turn consists of the following messages, and will accumulate with every new turn.

- The USER message
- Followed by the CHATBOT message with the list of tool calls
- Followed the TOOL message with the list of tool results
- Finally, followed by the CHATBOT message with the final response to the user


In [ ]:
chat_history = run_assistant("Considering the new appointments you made, when is my next available time?", chat_history)

In [ ]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")